In [1]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import vtk as vtk

In [2]:
%matplotlib widget

# Try to load case with `pyvista.read`

PyVista is smart enough to allow the load of an OpenFOAM case (`.foam` extension here)

In [3]:
# Import the OpenFOAM example case
case = pv.read('./case.foam')

In [4]:
case.keys()

['internalMesh']

In [5]:
mesh = case[0]

In [6]:
mesh

UnstructuredGrid (0x7fd7336eb040)
  N Cells:	12225
  N Points:	25012
  X Bounds:	-2.060e-02, 2.900e-01
  Y Bounds:	-2.540e-02, 2.540e-02
  Z Bounds:	-5.000e-04, 5.000e-04
  N Arrays:	10

* It looks like the number of arrays is doubled here, there is 2 times the `Data Arrays` (don't know why)
* It looks like the case is loaded for `time=0`, I don't know how to change the time value.

Here is for example below the U field where it's 0 everywhere expect at the inlet which is the intial condtion at `time=0`.

In [7]:
mesh.plot(scalars='U',cpos='xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Load the case with `vtkOpenFOAMReader`

In [8]:
reader = vtk.vtkOpenFOAMReader()
reader.SetFileName("case.foam")
reader.UpdateInformation()
time_array = reader.GetTimeValues()
reader.SetTimeValue(time_array.GetRange()[1])  # Set read time to latest time step
reader.Update()
multi_block = pv.wrap(reader.GetOutput())
mesh = multi_block[0]

## Let's make a simple contour plot

Let's plot the $u_x$ component of the velocity field

In [9]:
mesh.plot(scalars='U',component=0, cpos='xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Let's use a different and more useful API

### Show the mesh 

In [10]:
# Create a plotting object to display vtk meshes or numpy arrays
p1 = pv.Plotter()
# Show the mesh in wireframe style
p1.add_mesh(mesh, style="wireframe", color="w")
# View in the xy plane
p1.view_xy()
# Show the axes
p1.add_axes()
# Display the plotting window
p1.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### An example with a field contour

In [11]:
p2 = pv.Plotter()
p2.add_mesh(mesh,scalars='U')
p2.view_xy()
p2.add_axes()
p2.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Another one with the mesh surimposed

In [12]:
p3 = pv.Plotter()
p3.add_mesh(mesh,scalars='U', show_edges=True)
p3.view_xy()
p3.add_axes()
p3.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Plot over line example

In [13]:
# Create the 2 points defining the line
a = [0, 0, 0]
b = [0, mesh.bounds[3], 0]

# Create a Line object
line = pv.Line(a, b)

Let's show the line in the computational domain

In [14]:
p4 = pv.Plotter()
p4.add_mesh(mesh, style="wireframe", color='white')
# Show the line in red in the domain
p4.add_mesh(line, color="r", line_width=10)
p4.view_xy()
p4.add_axes()
p4.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Let's make a plot overline ...

In [15]:
mesh.plot_over_line(a, b, resolution=100, scalars='U')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### How to access the vector components

List the available arrays 

In [16]:
mesh.cell_arrays

pyvista DataSetAttributes
Association: CELL
Contains keys:
	U
	epsilon
	k
	nut
	p

Let's put the velocity vector field in `U`

In [17]:
U = mesh.cell_arrays[0]
np.shape(U)

(12225, 3)

Put each component in numpy arrays

In [18]:
ux = U[:,0]
uy = U[:,1]
uz = U[:,2]

# One can also use
ux = mesh['U'][:,0]
uy = mesh['U'][:,1]
uz = mesh['U'][:,2]

In [19]:
mesh.cell_arrays["ux"] = ux
mesh.cell_arrays["uy"] = uy
mesh.cell_arrays["uz"] = uz

In [20]:
mesh.plot_over_line(a, b, resolution=50, scalars='ux')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot a scalar over a circular arc

In [21]:
# Make two points at the bounds of the mesh and one at the center to
# construct a circular arc.
normal = [0, 0, 1]
polar = [0.02, 0, 0]
center = [0, 0, 0]
angle = 100.0

# Preview how this circular arc intersects this mesh
arc = pv.CircularArcFromNormal(center, 100, normal, polar, angle)

p = pv.Plotter()
p.add_mesh(mesh, style="wireframe", color="w")
p.add_mesh(arc, color="r")
a = arc.points[0]
b = arc.points[-1]
p.add_point_labels(
    [a, b], ["A", "B"], font_size=48, point_color="red", text_color="red"
)
p.view_xy()
p.show()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Let's see what's in the arc object.

In [22]:
arc

PolyData (0x7fd75d737400)
  N Cells:	1
  N Points:	101
  X Bounds:	-3.473e-03, 2.000e-02
  Y Bounds:	0.000e+00, 2.000e-02
  Z Bounds:	0.000e+00, 0.000e+00
  N Arrays:	2

I don't know how to add another plot on the same figure, it's perhaps only possible to plot one scalar or one vector at a time with this API.

In [23]:
mesh.plot_over_circular_arc_normal(center, 100, normal, polar, angle,scalars='ux')
#mesh.plot_over_circular_arc_normal(center, 100, normal, polar, angle, scalars='uy')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So let's try another way and create the figure ourself (there may be better ways to do this by the way).

In [24]:
sampled = pv.DataSetFilters.sample_over_circular_arc_normal(mesh,
                                                         center,
                                                         100,
                                                         normal,
                                                         polar,
                                                         angle,
                                                         )


In [25]:
plt.figure()
plt.plot(sampled['Distance'],sampled['ux'],label='$u_x$')
plt.plot(sampled['Distance'],sampled['uy'],label='$u_x$')
plt.xlabel('Arc distance')
plt.ylabel('Velocity')
plt.title('Distribution of the velocity components along the arc')
plt.grid(linestyle='dotted')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …